## The Logistics Commander (CoT & ToT)

In [1]:
import pandas as pd
import sys
sys.path.append('..')

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model, should_use_reasoning_model
from IPython.display import Markdown, display

In [2]:
file_path = "../data/incidents.txt"

# Read file as raw text
with open(file_path, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

# First line is header
header = lines[0]
rows = lines[1:]

rows

['1  | 08:00 AM| Gampaha | 4      | 20-40  | Water     | "Thirsty but safe on roof. Water level stable."',
 '2  | 08:15 AM| Ja-Ela  | 1      | 75     | Insulin   | "Diabetic, missed dose yesterday. Feeling faint."',
 '3  | 08:20 AM| Ragama  | 2      | 10, 35 | Rescue    | "Water approaching neck level. Child is crying."']

In [3]:
incidents = []

for row in rows:
    parts = [p.strip() for p in row.split("|")]

    incident = {
        "id": int(parts[0]),
        "time": parts[1],
        "area": parts[2],
        "people": int(parts[3]),
        "ages": parts[4],
        "main_need": parts[5],
        "message": parts[6].strip('"')
    }

    incidents.append(incident)

incidents


[{'id': 1,
  'time': '08:00 AM',
  'area': 'Gampaha',
  'people': 4,
  'ages': '20-40',
  'main_need': 'Water',
  'message': 'Thirsty but safe on roof. Water level stable.'},
 {'id': 2,
  'time': '08:15 AM',
  'area': 'Ja-Ela',
  'people': 1,
  'ages': '75',
  'main_need': 'Insulin',
  'message': 'Diabetic, missed dose yesterday. Feeling faint.'},
 {'id': 3,
  'time': '08:20 AM',
  'area': 'Ragama',
  'people': 2,
  'ages': '10, 35',
  'main_need': 'Rescue',
  'message': 'Water approaching neck level. Child is crying.'}]

In [4]:
df_incidents = pd.DataFrame(incidents)
df_incidents

,id,time,area,people,ages,main_need,message
0,1,08:00 AM,Gampaha,4,20-40,Water,Thirsty but safe on roof. Water level stable.
1,2,08:15 AM,Ja-Ela,1,75,Insulin,"Diabetic, missed dose yesterday. Feeling faint."
2,3,08:20 AM,Ragama,2,"10, 35",Rescue,Water approaching neck level. Child is crying.


In [5]:
def parse_ages(age_str):
    nums = [int(x) for x in age_str.replace("-", ",").split(",")]
    return min(nums), max(nums)

df_incidents[["min_age", "max_age"]] = df_incidents["ages"].apply(
    lambda x: pd.Series(parse_ages(x))
)

df_incidents

,id,time,area,people,ages,main_need,message,min_age,max_age
0,1,08:00 AM,Gampaha,4,20-40,Water,Thirsty but safe on roof. Water level stable.,20,40
1,2,08:15 AM,Ja-Ela,1,75,Insulin,"Diabetic, missed dose yesterday. Feeling faint.",75,75
2,3,08:20 AM,Ragama,2,"10, 35",Rescue,Water approaching neck level. Child is crying.,10,35


In [6]:
# CoT auto-routes to reasoning model
reasoning_model = pick_model('groq', 'cot')
print(f'Using reasoning model: {reasoning_model}')

client_reasoning = LLMClient('groq', reasoning_model)


problem = incidents

# Additional guidance 
instruction = """You are a disaster logistics analyst.

You must evaluate the following incident and assign a
priority score using step-by-step reasoning internally.

-----------------------------------
SCORING RULES:
- Base Score: 5
- +2 if any person is younger than 5 or older than 60
- +3 if Main Need is "Rescue" (immediate life threat)
- +1 if Main Need is "Insulin" or urgent medicine
- Maximum possible score: 10

-----------------------------------
INCIDENT DETAILS:
Area: {area}
People: {people}
Ages: {ages}
Main Need: {main_need}
Message: "{message}"

-----------------------------------
IMPORTANT:
- You MUST reason step-by-step internally
- You MUST NOT show your reasoning
- You MUST output ONLY the final score

-----------------------------------
OUTPUT FORMAT (STRICT):
{row['id']} | {row['area']} | {row['time']} | {row['people']} | {row['main_need']} | {row['message']} | SCORE}
				
"""

prompt_text, spec = render(
    'cot_reasoning.v1',
    role='disaster logistics analyst',
    problem=problem
)

# Combine problem with instruction 
full_prompt = f"""text: {prompt_text}

instruction: {instruction}"""

messages = [{'role': 'user', 'content': full_prompt}]
response = client_reasoning.chat(messages, temperature=spec.temperature, max_tokens=spec.max_tokens)

print('CoT Response :')
print('=' * 80)
print(response['text'])
print('=' * 80)
log_llm_call('groq', reasoning_model, 'cot', response['latency_ms'], response['usage'])

Using reasoning model: openai/gpt-oss-120b
CoT Response :
1 | Gampaha | 08:00 AM | 4 | Water | Thirsty but safe on roof. Water level stable. | 5
2 | Ja-Ela | 08:15 AM | 1 | Insulin | Diabetic, missed dose yesterday. Feeling faint. | 8
3 | Ragama | 08:20 AM | 2 | Rescue | Water approaching neck level. Child is crying. | 8


In [7]:
incident_table = f"ID | Area | Time | People | Main Need | Message | Priority Score\n{response['text']}"

print(incident_table)

ID | Area | Time | People | Main Need | Message | Priority Score
1 | Gampaha | 08:00 AM | 4 | Water | Thirsty but safe on roof. Water level stable. | 5
2 | Ja-Ela | 08:15 AM | 1 | Insulin | Diabetic, missed dose yesterday. Feeling faint. | 8
3 | Ragama | 08:20 AM | 2 | Rescue | Water approaching neck level. Child is crying. | 8


In [11]:
problem = f"""You have ONE rescue boat located at Ragama.

Travel Time Constraints:
- Ragama → Ja-Ela: 10 minutes
- Ja-Ela → Gampaha: 40 minutes

Each incident requires one stop.
Assume unlimited capacity per stop.

-----------------------------------
INCIDENTS WITH PRIORITY SCORES:
{incident_table}

-----------------------------------
STRATEGY BRANCHES TO EXPLORE:

Branch 1 - Greedy (Highest Score First):
Visit incidents in descending order of priority score.

Branch 2 - Speed (Closest First):
Visit the nearest locations first to minimize travel time.

Branch 3 - Logistics (Furthest First):
Visit the furthest locations first to reduce later delays.

-----------------------------------
GOAL:
Maximize total priority score saved within the shortest
overall travel time.

-----------------------------------
TASK:
1. Evaluate each branch independently
2. Compute total score saved and travel time
3. Compare all branches
4. Select the optimal route

-----------------------------------
OUTPUT FORMAT:

Branch 1 Analysis:
...

Branch 2 Analysis:
...

Branch 3 Analysis:
...

FINAL DECISION:
Optimal Route: ...
Justification: ...
"""

prompt_text, spec = render(
    'tot_reasoning.v1',
    role='disaster logistics analyst',
    problem=problem,
    branches='3'
)

messages = [{'role': 'user', 'content': prompt_text}]
response = client_reasoning.chat(messages, temperature=spec.temperature, max_tokens=spec.max_tokens)

print('ToT Response (Multiple Solution Paths):')
print('=' * 80)
display(Markdown(response['text']))
print('=' * 80)
log_llm_call('groq', reasoning_model, 'tot', response['latency_ms'], response['usage'])

ToT Response (Multiple Solution Paths):


**Branch 1 – Greedy (Highest Score First)**  

**Hypothesis** – By tackling the two incidents with the highest priority scores (8 pts each) before the lower‑scoring one (5 pts), we will maximise the “score saved” even if the travel distance grows.  

**Steps**  

1. **Start @ Ragama (08:00).**  
2. **Go to Ja‑Ela** – 10 min travel → arrive 08:10 (before the 08:15 incident).  
   *Serve ID 2 (score 8).*  
3. **Return to Ragama** – 10 min travel → arrive 08:20 (exactly when ID 3 needs help).  
   *Serve ID 3 (score 8).*  
4. **Proceed to Gampaha** – Ragama → Ja‑Ela (10 min) + Ja‑Ela → Gampaha (40 min) = 50 min → arrive 09:10.  
   *Serve ID 1 (score 5).*

**Intermediate check** – All three incidents are eventually served; total travel = 10 + 10 + 50 = **70 min**.  

**Result** – Total priority score saved = **8 + 8 + 5 = 21** points.  

---

**Branch 2 – Speed (Closest First)**  

**Hypothesis** – Visiting the nearest locations first minimises dead‑head travel, allowing us to finish the whole mission faster while still rescuing everyone.  

**Steps**  

1. **Start @ Ragama (08:00).**  
2. **Serve Ragama incident immediately** (no travel).  
   *Serve ID 3 at 08:20 (score 8).*  
3. **Go to Ja‑Ela** – 10 min travel → arrive 08:30 (15 min after the incident time, but still reachable).  
   *Serve ID 2 (score 8).*  
4. **Go to Gampaha** – 40 min travel → arrive 09:10.  
   *Serve ID 1 (score 5).*

**Intermediate check** – All incidents are covered; total travel = **0 + 10 + 40 = 50 min**.  

**Result** – Total priority score saved = **21** points (same as Branch 1) with **20 min less travel**.

---

**Branch 3 – Logistics (Furthest First)**  

**Hypothesis** – By heading to the farthest point first we eliminate the “return‑trip penalty” later; the longer leg is done only once.  

**Steps**  

1. **Start @ Ragama (08:00).**  
2. **Travel to Gampaha** – Ragama → Ja‑Ela (10 min) + Ja‑Ela → Gampaha (40 min) = 50 min → arrive 08:50.  
   *Serve ID 1 (score 5).*  
3. **Backtrack to Ja‑Ela** – 40 min travel → arrive 09:30.  
   *Serve ID 2 (score 8).*  
4. **Return to Ragama** – 10 min travel → arrive 09:40.  
   *Serve ID 3 (score 8).*

**Intermediate check** – All incidents are served; total travel = **50 + 40 + 10 = 100 min**.  

**Result** – Total priority score saved = **21** points, but with the longest overall travel time.

---

### Comparison

| Branch | Order of Visits | Travel Time | Score Saved |
|--------|----------------|------------|------------|
| 1 – Greedy | Ja‑Ela → Ragama → Gampaha | 70 min | 21 |
| 2 – Speed  | Ragama → Ja‑Ela → Gampaha | **50 min** | **21** |
| 3 – Logistics | Gampaha → Ja‑Ela → Ragama | 100 min | 21 |

All three routes ultimately rescue every victim, yielding the same total priority score (21 pts). The decisive factor is **overall travel time**.  

---

## FINAL DECISION  
**Optimal Route:** **Branch 2 – Speed (Closest First)** → **Ragama → Ja‑Ela → Gampaha**  

**Justification:**  
- Saves the full 21 priority points (no loss of lives/needs).  
- Minimises total travel to **50 minutes**, 20 minutes faster than the Greedy approach and 50 minutes faster than the Logistics approach.  
- By handling the incident already at the boat’s location first, we avoid any unnecessary waiting and keep the mission as swift as possible, which is critical in disaster response.  

**Answer:** Deploy the rescue boat to **Ragama (serve incident 3), then sail 10 min to Ja‑Ela (serve incident 2), then continue 40 min to Gampaha (serve incident 1)** – total travel 50 minutes, total priority score saved = 21.